In [ ]:
import pandas as pd
import geopandas as gpd
import geemap as gm
import os
import json
import geojson
import requests

In [ ]:
import ee

ee.Authenticate()
ee.Initialize(project="ee-forestplotvariables")

In [ ]:
import ipywidgets as widgets

start_date = widgets.DatePicker(description="Start Date", disabled=False)
start_date

In [ ]:
end_date = widgets.DatePicker(description="Start Date", disabled=False)
end_date

In [ ]:
date = str(end_date.value)
type(date)
print(date)

In [ ]:
dataset_id = f"USGS/3DEP/10m"
dataset = ee.ImageCollection(dataset_id)
if isinstance(dataset_id, ee.Image):
    print("This is an Image")
elif isinstance(dataset_id, ee.ImageCollection):
    print("This is an ImageCollection")
else:
    print("Unknown type")

In [ ]:
dataset_id = f"IDAHO_EPSCOR/TERRACLIMATE"
dataset = ee.ImageCollection(dataset_id)
m = gm.Map()
m.set_center(-122.503881, 37.765588, 18)
m.add_layer(dataset, {"bands": ["tmmn", "tmmx", "pr"], "min": -800, "max": 800}, "img")
# display(m)

In [ ]:
path = "../data/5000.0.geojson"
# data = gpd.read_file(path)
with open(path, "r") as file:
    data = geojson.load(file)
dataset_id = f"IDAHO_EPSCOR/TERRACLIMATE"
start_date = None
end_date = None

out_dir = os.path.join(os.path.expanduser("~"), "Downloads")
output_file = f"test.csv"
out_path = os.path.join(out_dir, output_file)

# Convert GeoDataFrame to GeoJSON dictionary (to feed into Earth Engine)
geojson_object = gm.geojson_to_ee(data)

url = "https://raw.githubusercontent.com/opengeos/geospatial-data-catalogs/master/gee_catalog.json"
response = requests.get(url)
response.raise_for_status()  # Raises an exception for HTTP errors
geojson_data = response.json()

data_type = [item["type"] for item in geojson_data if item["id"] == dataset_id]
data_str = " ".join(data_type)

if data_str == "image":
    img = ee.Image(dataset_id)

results = img.sampleRegions(collection=geojson_object, geometries=True, scale=10)

results = results.aggregate_stats("").getInfo()

In [ ]:
path = "../data/10.0.geojson"
# data = geojson.load(file)
with open(path, "r") as file:
    data = geojson.load(file)
dataset_id = f"IDAHO_EPSCOR/TERRACLIMATE"
start_date = None
end_date = None

out_dir = os.path.join(os.path.expanduser("~"), "Downloads")
output_file = f"test.csv"
out_path = os.path.join(out_dir, output_file)

# Convert GeoDataFrame to GeoJSON dictionary (to feed into Earth Engine)
geojson_object = gm.geojson_to_ee(data)

url = "https://raw.githubusercontent.com/opengeos/geospatial-data-catalogs/master/gee_catalog.json"
response = requests.get(url)
response.raise_for_status()  # Raises an exception for HTTP errors
geojson_data = response.json()

data_type = [item["type"] for item in geojson_data if item["id"] == dataset_id]
data_str = " ".join(data_type)

if data_str == "image":
    img = ee.Image(dataset_id)
    median_values = img.reduceRegion(
        reducer=ee.Reducer.median(),  # median pixel value of each band within the polygon region
        geometry=geojson_object,  # your ee.Geometry polygon
        scale=30,  # suitable for your image
        maxPixels=1e9,
    ).getInfo()

    feature = ee.Feature(geojson_object, median_values)
    fc = ee.FeatureCollection([feature])
    gm.ee_to_csv(fc, filename=out_path)


elif data_str == "image_collection":
    col = ee.ImageCollection(dataset_id)
    if start_date and end_date:
        col = col.filterDate(start_date, end_date)
    else:
        pass
    col = col.filterBounds(geojson_object)
    # Reduce to median image
    median_image = col.reduce(ee.Reducer.median())
    median_values = median_image.reduceRegion(
        reducer=ee.Reducer.mean(),  # average pixel value of the median image within polygon
        geometry=geojson_object.geometry(),  # or .geometry() if FeatureCollection
        scale=30,  # set scale appropriate to data
        maxPixels=1e9,
    )  # .getInfo()

    # feature = ee.Feature(geojson_object, median_values)
    # fc = ee.FeatureCollection([feature])
    # gm.ee_to_csv(fc, filename=out_path)

    task = ee.batch.Export.table.toDrive(
        collection=median_values,
        description="median_band_export",
        folder="ee-demo",
        fileFormat="GeoJSON",
        selectors=".geo",
    )
task.start()

# Try loading as FeatureCollection (convert to raster mask)
# else:
#     print("Selected dataset is not an image or image collection, and could not be processed.")

In [ ]:
print(fc)

In [ ]:
print(type(results))
pd.DataFrame.from_dict(results, orient="index")